In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

yr2020_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv')
yr2020_df['fips'] = yr2020_df['fips'].fillna(0).astype(np.int64)
yr2020_df['date'] = pd.to_datetime(yr2020_df['date'], yearfirst=True)
yr2021_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv')
yr2021_df['fips'] = yr2021_df['fips'].fillna(0).astype(np.int64)
yr2021_df['date'] = pd.to_datetime(yr2021_df['date'], yearfirst=True)
yr2022_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv')
yr2022_df['fips'] = yr2022_df['fips'].fillna(0).astype(np.int64)
yr2022_df['date'] = pd.to_datetime(yr2022_df['date'], yearfirst=True)
us_covid_df = pd.concat([yr2020_df, yr2021_df, yr2022_df])
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
2655899,2022-08-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2655900,2022-08-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2655901,2022-08-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
2655902,2022-08-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
67827,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
67829,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
67831,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
67833,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
67835,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1604205,2022-08-03,Arkansas,Arkansas,5001,6191,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1604207,2022-08-04,Arkansas,Arkansas,5001,6202,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1604209,2022-08-05,Arkansas,Arkansas,5001,6212,79.00,5000,Arkansas,AR,Arkansas,State,3017804
1604211,2022-08-06,Arkansas,Arkansas,5001,6220,79.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2655899,2022-08-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2655900,2022-08-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2655901,2022-08-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
2655902,2022-08-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-08-07') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
929,2022-08-07,Snohomish,Washington,184142,1235.00,53061,WA,County,822083,22399.44,150.23
1856,2022-08-07,Cook,Illinois,1347608,15180.00,17031,IL,County,5150233,26165.96,294.74
2782,2022-08-07,Orange,California,699194,7235.00,6059,CA,County,3175692,22017.06,227.82
3707,2022-08-07,Maricopa,Arizona,1392047,17664.00,4013,AZ,County,4485414,31034.97,393.81
4632,2022-08-07,Los Angeles,California,3326326,32826.00,6037,CA,County,10039107,33133.68,326.98
...,...,...,...,...,...,...,...,...,...,...,...
2653371,2022-08-07,Wheeler,Oregon,258,3.00,41069,OR,County,1332,19369.37,225.23
2654035,2022-08-07,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00
2654668,2022-08-07,Esmeralda,Nevada,118,4.00,32009,NV,County,873,13516.61,458.19
2655297,2022-08-07,Loving,Texas,301,1.00,48301,TX,County,169,178106.51,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
929,2022-08-07,Snohomish,Washington,184142,1235.00,53061,WA,County,822083,22399.44,150.23,150.23,22399.44
1856,2022-08-07,Cook,Illinois,1347608,15180.00,17031,IL,County,5150233,26165.96,294.74,294.74,26165.96
2782,2022-08-07,Orange,California,699194,7235.00,6059,CA,County,3175692,22017.06,227.82,227.82,22017.06
3707,2022-08-07,Maricopa,Arizona,1392047,17664.00,4013,AZ,County,4485414,31034.97,393.81,393.81,31034.97
4632,2022-08-07,Los Angeles,California,3326326,32826.00,6037,CA,County,10039107,33133.68,326.98,326.98,33133.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2653371,2022-08-07,Wheeler,Oregon,258,3.00,41069,OR,County,1332,19369.37,225.23,225.23,19369.37
2654035,2022-08-07,King,Texas,52,0.00,48269,TX,County,272,19117.65,0.00,0.00,19117.65
2654668,2022-08-07,Esmeralda,Nevada,118,4.00,32009,NV,County,873,13516.61,458.19,458.19,13516.61
2655297,2022-08-07,Loving,Texas,301,1.00,48301,TX,County,169,178106.51,591.72,591.72,178106.51


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1678342,2022-08-07,Hale,Alabama,5190,109.00,1065,AL,County,14651,35424.20,743.98,743.98,35424.20,1
1415106,2022-08-07,Winston,Alabama,8127,134.00,1133,AL,County,23629,34394.18,567.10,567.10,34394.18,2
1156305,2022-08-07,Clay,Alabama,4474,83.00,1027,AL,County,13235,33804.31,627.13,627.13,33804.31,3
1045334,2022-08-07,Morgan,Alabama,39913,524.00,1103,AL,County,119679,33350.04,437.84,437.84,33350.04,4
971556,2022-08-07,Franklin,Alabama,10417,140.00,1059,AL,County,31362,33215.36,446.40,446.40,33215.36,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600764,2022-08-07,Converse,Wyoming,3351,55.00,56009,WY,County,13822,24243.96,397.92,397.92,24243.96,19
2268228,2022-08-07,Big Horn,Wyoming,2832,63.00,56003,WY,County,11790,24020.36,534.35,534.35,24020.36,20
1996249,2022-08-07,Lincoln,Wyoming,4540,32.00,56023,WY,County,19830,22894.60,161.37,161.37,22894.60,21
1601627,2022-08-07,Sublette,Wyoming,2110,27.00,56035,WY,County,9831,21462.72,274.64,274.64,21462.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1309581,2022-08-07,Lowndes,Alabama,2912,79.00,1085,AL,County,9726,29940.37,812.26,812.26,29940.37,26,1
1678342,2022-08-07,Hale,Alabama,5190,109.00,1065,AL,County,14651,35424.20,743.98,743.98,35424.20,1,2
1307851,2022-08-07,Crenshaw,Alabama,4167,102.00,1041,AL,County,13772,30257.04,740.63,740.63,30257.04,23,3
552806,2022-08-07,Walker,Alabama,20542,453.00,1127,AL,County,63521,32338.91,713.15,713.15,32338.91,8,4
1158903,2022-08-07,Dallas,Alabama,9598,249.00,1047,AL,County,37196,25803.85,669.43,669.43,25803.85,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410786,2022-08-07,Johnson,Wyoming,2082,20.00,56019,WY,County,8445,24653.64,236.83,236.83,24653.64,18,19
1945686,2022-08-07,Uinta,Wyoming,5991,39.00,56041,WY,County,20226,29620.29,192.82,192.82,29620.29,8,20
1996249,2022-08-07,Lincoln,Wyoming,4540,32.00,56023,WY,County,19830,22894.60,161.37,161.37,22894.60,21,21
1304391,2022-08-07,Albany,Wyoming,10809,51.00,56001,WY,County,38880,27800.93,131.17,131.17,27800.93,14,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,150.23,22399.44,32,16
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,150.23,22399.44,32,16
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,150.23,22399.44,32,16
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,150.23,22399.44,32,16
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,150.23,22399.44,32,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649795,2022-08-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2649796,2022-08-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2649797,2022-08-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
2649798,2022-08-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1671378,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,743.98,35424.20,2,1,1.00,nan
1671379,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,743.98,35424.20,2,1,0.00,nan
1671380,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,743.98,35424.20,2,1,0.00,nan
1671381,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,743.98,35424.20,2,1,0.00,nan
1671382,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,743.98,35424.20,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030373,2022-08-03,Crook,Wyoming,1445,29.00,56011,WY,County,7584,19053.27,382.38,382.38,19053.27,10,23,0.00,1.43
2030374,2022-08-04,Crook,Wyoming,1445,29.00,56011,WY,County,7584,19053.27,382.38,382.38,19053.27,10,23,0.00,1.43
2030375,2022-08-05,Crook,Wyoming,1445,29.00,56011,WY,County,7584,19053.27,382.38,382.38,19053.27,10,23,0.00,1.43
2030376,2022-08-06,Crook,Wyoming,1445,29.00,56011,WY,County,7584,19053.27,382.38,382.38,19053.27,10,23,0.00,1.43


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa['14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa['deaths'] = usa['deaths'].astype(int)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1304339,2020-03-26,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,812.26,29940.37,1,26,1.00,9.57,0.00,nan
1304340,2020-03-27,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,812.26,29940.37,1,26,0.00,8.29,0.00,nan
1304341,2020-03-28,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,812.26,29940.37,1,26,0.00,7.21,0.00,nan
1304342,2020-03-29,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,812.26,29940.37,1,26,0.00,6.64,0.00,nan
1304343,2020-03-30,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,812.26,29940.37,1,26,0.00,5.50,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546674,2022-08-03,Teton,Wyoming,11304,16,56039,WY,County,23464,48175.93,68.19,68.19,48175.93,23,1,0.00,11.14,0.00,0.00
546675,2022-08-04,Teton,Wyoming,11304,16,56039,WY,County,23464,48175.93,68.19,68.19,48175.93,23,1,0.00,11.14,0.00,0.00
546676,2022-08-05,Teton,Wyoming,11304,16,56039,WY,County,23464,48175.93,68.19,68.19,48175.93,23,1,0.00,11.14,0.00,0.00
546677,2022-08-06,Teton,Wyoming,11304,16,56039,WY,County,23464,48175.93,68.19,68.19,48175.93,23,1,0.00,11.14,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-08-07') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
660902,2022-08-07,Imperial,California,77352,949,6025,CA,County,181215,42685.21,523.69,523.69,42685.21,1,1,0.00,147.50,0.00,0.07
100512,2022-08-07,Shasta,California,40674,670,6089,CA,County,180080,22586.63,372.06,372.06,22586.63,2,36,0.00,41.71,0.00,0.43
327256,2022-08-07,San Bernardino,California,667415,7956,6071,CA,County,2180085,30614.17,364.94,364.94,30614.17,3,7,0.00,921.64,0.00,1.64
1602419,2022-08-07,Tuolumne,California,16071,196,6109,CA,County,54478,29499.98,359.78,359.78,29499.98,4,9,0.00,25.86,0.00,0.00
4632,2022-08-07,Los Angeles,California,3326326,32826,6037,CA,County,10039107,33133.68,326.98,326.98,33133.68,5,4,0.00,5164.21,0.00,15.86
180654,2022-08-07,Tulare,California,150663,1516,6107,CA,County,466195,32317.59,325.19,325.19,32317.59,6,5,0.00,127.79,0.00,0.21
1944724,2022-08-07,Tehama,California,16071,209,6103,CA,County,65084,24692.70,321.12,321.12,24692.70,7,25,0.00,13.64,0.00,0.14
1163185,2022-08-07,Inyo,California,5209,57,6027,CA,County,18039,28876.32,315.98,315.98,28876.32,8,11,0.00,4.50,0.00,0.00
145457,2022-08-07,Stanislaus,California,152202,1701,6099,CA,County,550660,27639.92,308.90,308.90,27639.92,9,15,0.00,180.14,0.00,0.43
1053093,2022-08-07,Merced,California,79923,838,6047,CA,County,277680,28782.41,301.79,301.79,28782.41,10,12,0.00,105.21,0.00,0.07


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
660902,2022-08-07,Imperial,California,77352,949,6025,CA,County,181215,42685.21,523.69,523.69,42685.21,1,1,0.00,147.50,0.00,0.07
1413320,2022-08-07,Kings,California,61168,453,6031,CA,County,152940,39994.77,296.19,296.19,39994.77,12,2,0.00,79.86,0.00,0.14
2495131,2022-08-07,Lassen,California,11295,68,6035,CA,County,30573,36944.36,222.42,222.42,36944.36,21,3,0.00,3.64,0.00,0.00
4632,2022-08-07,Los Angeles,California,3326326,32826,6037,CA,County,10039107,33133.68,326.98,326.98,33133.68,5,4,0.00,5164.21,0.00,15.86
180654,2022-08-07,Tulare,California,150663,1516,6107,CA,County,466195,32317.59,325.19,325.19,32317.59,6,5,0.00,127.79,0.00,0.21
66058,2022-08-07,Madera,California,49430,372,6039,CA,County,157327,31418.64,236.45,236.45,31418.64,19,6,0.00,75.14,0.00,0.00
327256,2022-08-07,San Bernardino,California,667415,7956,6071,CA,County,2180085,30614.17,364.94,364.94,30614.17,3,7,0.00,921.64,0.00,1.64
412173,2022-08-07,Kern,California,272015,2499,6029,CA,County,900202,30217.11,277.60,277.60,30217.11,15,8,0.00,316.64,0.00,1.07
1602419,2022-08-07,Tuolumne,California,16071,196,6109,CA,County,54478,29499.98,359.78,359.78,29499.98,4,9,0.00,25.86,0.00,0.00
65174,2022-08-07,Fresno,California,291838,2815,6019,CA,County,999101,29210.06,281.75,281.75,29210.06,13,10,0.00,468.86,0.00,1.64


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
660902,2022-08-07,Imperial,California,77352,949,6025,CA,County,181215,42685.21,523.69,523.69,42685.21,1,1,0.00,147.50,0.00,0.07
100512,2022-08-07,Shasta,California,40674,670,6089,CA,County,180080,22586.63,372.06,372.06,22586.63,2,36,0.00,41.71,0.00,0.43
327256,2022-08-07,San Bernardino,California,667415,7956,6071,CA,County,2180085,30614.17,364.94,364.94,30614.17,3,7,0.00,921.64,0.00,1.64
1602419,2022-08-07,Tuolumne,California,16071,196,6109,CA,County,54478,29499.98,359.78,359.78,29499.98,4,9,0.00,25.86,0.00,0.00
4632,2022-08-07,Los Angeles,California,3326326,32826,6037,CA,County,10039107,33133.68,326.98,326.98,33133.68,5,4,0.00,5164.21,0.00,15.86
180654,2022-08-07,Tulare,California,150663,1516,6107,CA,County,466195,32317.59,325.19,325.19,32317.59,6,5,0.00,127.79,0.00,0.21
1944724,2022-08-07,Tehama,California,16071,209,6103,CA,County,65084,24692.70,321.12,321.12,24692.70,7,25,0.00,13.64,0.00,0.14
1163185,2022-08-07,Inyo,California,5209,57,6027,CA,County,18039,28876.32,315.98,315.98,28876.32,8,11,0.00,4.50,0.00,0.00
145457,2022-08-07,Stanislaus,California,152202,1701,6099,CA,County,550660,27639.92,308.90,308.90,27639.92,9,15,0.00,180.14,0.00,0.43
1053093,2022-08-07,Merced,California,79923,838,6047,CA,County,277680,28782.41,301.79,301.79,28782.41,10,12,0.00,105.21,0.00,0.07


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df['case_day'] = top10_ca_case_df['case_day'].astype(int)
top10_ca_case_df['date'] =  pd.to_datetime(top10_ca_case_df['date'], infer_datetime_format=True)
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
0,2020-03-20,Imperial,4,2.21,4,1.43
1,2020-03-21,Imperial,4,2.21,0,1.14
2,2020-03-22,Imperial,4,2.21,0,1.14
3,2020-03-23,Imperial,4,2.21,0,1.00
4,2020-03-24,Imperial,9,4.97,5,1.29
...,...,...,...,...,...,...
8721,2022-08-03,Lassen,11290,36928.01,0,3.29
8722,2022-08-04,Lassen,11295,36944.36,5,3.64
8723,2022-08-05,Lassen,11295,36944.36,0,3.64
8724,2022-08-06,Lassen,11295,36944.36,0,3.64


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df['death_day'] = top10_ca_death_df['death_day'].astype(int)
top10_ca_death_df['date'] =  pd.to_datetime(top10_ca_death_df['date'], infer_datetime_format=True)
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,2020-03-20,Imperial,0,0.00,0,0.00
1,2020-03-21,Imperial,0,0.00,0,0.00
2,2020-03-22,Imperial,0,0.00,0,0.00
3,2020-03-23,Imperial,0,0.00,0,0.00
4,2020-03-24,Imperial,0,0.00,0,0.00
...,...,...,...,...,...,...
8760,2022-08-03,Merced,838,301.79,0,0.36
8761,2022-08-04,Merced,838,301.79,0,0.36
8762,2022-08-05,Merced,838,301.79,0,0.07
8763,2022-08-06,Merced,838,301.79,0,0.07


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)